In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
#
import os
import random
from textwrap import dedent

from gait import FEL, Layers, s_message, u_message, a_message
from litellm import completion
from rich.pretty import pprint

In [4]:
layers = Layers.load(os.path.expanduser("~/data/NorthSea.json"))
fel = FEL(layers=layers.layers)

In [5]:
attributes = fel.attributes()
# print(attributes)

In [6]:
# model = "qwen2:7b-instruct-q8_0"
# model = "dolphin3:8b-llama3.1-q8_0"
model = "ollama_chat/phi4:14b-q8_0"
# model = "command-r7b:7b-12-2024-q8_0"
# model = f"azure/{os.environ['AZURE_API_DEPLOYMENT']}"

In [7]:
system = dedent(
    f"""
You are an AI specialized in creating variations of geospatial and non-geospatial queries.
Your task is to generate one variation of a user's prompt, incorporating spatial relations, non-spatial relations, and spatial layer filter attributes as appropriate.
Adapt to the context while maintaining logical consistency and relevance to the original query.

Guidelines:
1. Analyze the original query to identify spatial and non-spatial elements.
2. Create ONE variation that rephrases the query with a distinct focus or intent.
3. Incorporate relevant spatial relations (e.g., "intersects," "within," "near"), non-spatial relations (e.g., attribute-based filtering, temporal aspects), and spatial layer filter attributes based on the provided schema.
4. Use clear, concise language in your variation.
5. Maintain the original query's core intent and logic.
6. Do not add spatial context if it doesn't exist in the original prompt.
7. Output only the variation without explanations.

If the query lacks clear spatial or non-spatial elements, focus on enhancing the existing elements or clarifying the intent.

Schema for spatial layers:
{attributes}

Respond with only the variation, without additional commentary.
"""
).strip()

In [8]:
print(system)

You are an AI specialized in creating variations of geospatial and non-geospatial queries.
Your task is to generate one variation of a user's prompt, incorporating spatial relations, non-spatial relations, and spatial layer filter attributes as appropriate.
Adapt to the context while maintaining logical consistency and relevance to the original query.

Guidelines:
1. Analyze the original query to identify spatial and non-spatial elements.
2. Create ONE variation that rephrases the query with a distinct focus or intent.
3. Incorporate relevant spatial relations (e.g., "intersects," "within," "near"), non-spatial relations (e.g., attribute-based filtering, temporal aspects), and spatial layer filter attributes based on the provided schema.
4. Use clear, concise language in your variation.
5. Maintain the original query's core intent and logic.
6. Do not add spatial context if it doesn't exist in the original prompt.
7. Output only the variation without explanations.

If the query lacks c

In [9]:
def variation(content: str) -> str:
    response = completion(
        model=model,
        temperature=0.35,
        messages=[
            s_message(system),
            u_message("Find pipelines where dimension < 20.0 that are inside wellbores"),
            a_message("Identify pipelines with dimensions less than 20.0 meters that intersect with wellbore locations"),
            u_message(
                "Identify all discoveries with discovery type > 3 and discovery current activity state is production ceased"),
            a_message(
                "Locate all discoveries where the discovery type exceeds 3 and the discovery's current activity status is marked as 'production ceased.'"),
            u_message("Find pipelines that are inside discoveries where field type is not like gas"),
            a_message("Locate pipelines within discoveries where the field type does not include 'gas'"),
            u_message(content),
        ],
    )
    return response.choices[0].message.content

In [10]:
line_fel = random.choice([fel.create_line_1, fel.create_line_2])()
# line_fel = fel.create_line_1()
print(line_fel.line)
pprint(line_fel, expand_all=True)
print()
print(variation(line_fel.line))

Identify pipelines where dimension is not 42.0 and dimension not exceeding 26.0 or pipe name <> 36" gas troll a, kollsnes


LineFEL(
│   line='Identify pipelines where dimension is not 42.0 and dimension not exceeding 26.0 or pipe name <> 36" gas troll a, kollsnes',
│   fel=FEL1(
│   │   layer='Pipelines',
│   │   where='dimension <> 42.0 and dimension < 26.0 or pipe_name <> \'36" Gas TROLL A, KOLLSNES\''
│   )
)


Find pipelines with dimensions that are neither exactly 42.0 nor greater than 26.0, excluding those named '36" gas troll A, Kollsnes.'
